## LLAMA INDEX INTEGRATION FOR NEURALDB

In [ ]:
# Install requiremnts
!pip install llama-index
!pip install thirdai[neural_db]
!pip install docx2txt

In [ ]:
import os
from thirdai import licensing

if "THIRDAI_KEY" in os.environ:
    licensing.activate(os.environ["THIRDAI_KEY"])
else:
    licensing.activate("")  # Enter your ThirdAI key here

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""  # Enter your OpenAI key here

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Path to the directory containing all the files
document_directory = "data/"

# load documents
documents = SimpleDirectoryReader(document_directory).load_data()

In [ ]:
from llama_index.core import Settings

nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import StorageContext

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [ ]:
from typing import List
from thirdai import neural_db as ndb
from llama_index.core.retrievers import (
    BaseRetriever,
)
# import NodeWithScore
from llama_index.core.schema import NodeWithScore

class NDBretriever(BaseRetriever):
    def __init__(self, nodes, storage_context, top_k=5):
        self.db = self.constructdb(nodes)
        self.storage_context = storage_context
        self.top_k = top_k
        
    def constructdb(self, nodes):
        db = ndb.NeuralDB()
        docs = []
        for node in nodes:
            doc = ndb.InMemoryText(name=node.node_id,texts=[node.text])
            docs.append(doc)
        
        db.insert(docs)
        
        return db

    def _retrieve(self, query_bundle):
        results = self.db.search(query_bundle.query_str, top_k=self.top_k)
        node_with_scores: List[NodeWithScore] = []
        for result in results:
            node = self.storage_context.docstore.get_node(result.source)
            node_with_scores.append(NodeWithScore(node=node, score=result.score))
        
        return node_with_scores

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# define custom retriever
ndb_retriever = NDBretriever(nodes=nodes,storage_context=storage_context,top_k=5)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=ndb_retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
response = custom_query_engine.query(
    "what is confidentiality?"
)
print(response)